In [3]:
import pandas as pd
from pymongo import MongoClient
from folium import Map,Marker

In [17]:
df = pd.read_csv('https://raw.githubusercontent.com/chrismeller/StarbucksLocations/master/stores.csv')
df.head(3)

,Id,StarbucksId,Name,BrandName,StoreNumber,PhoneNumber,OwnershipType,Street1,Street2,Street3,...,CountrySubdivisionCode,CountryCode,PostalCode,Longitude,Latitude,TimezoneOffset,TimezoneId,TimezoneOlsonId,FirstSeen,LastSeen
0,1f2d204f-e773-4361-9158-0008307dbd5e,10357,Target Virginia T-847,Starbucks,76666-97597,218-741-6603,LS,1001 13th St S,NaN,NaN,...,MN,US,557923254,-92.55,47.51,-360,Central Standard Time,GMT-06:00 America/Chicago,12/8/2013 5:41:59 PM,2/3/2017 12:00:00 AM
1,49455e22-49e9-461f-912c-00092386eaee,6557,PDX A 2,Starbucks,75916-104381,503-284-4008,LS,"7000 NE Airport Way, MB#3",NaN,NaN,...,OR,US,972181031,-122.59,45.59,-480,Pacific Standard Time,GMT-08:00 America/Los_Angeles,12/8/2013 5:41:59 PM,2/3/2017 12:00:00 AM
2,1316ebf8-ee4a-479d-a11d-000964af367b,1010600,235 Bourke Street,Starbucks,25294-240419,NaN,LS,235-251 Bourke St,"Shops 5,6 & 10",NaN,...,VIC,AU,3000,144.97,-37.81,660,AUS Eastern Standard Time,GMT+10:00 Australia/Melbourne,10/8/2014 12:00:00 AM,2/3/2017 12:00:00 AM


In [24]:
# filter Sbux in Canada as many of our offices
df = df[df['CountryCode']=='CA']
df = df[df['CountrySubdivisionCode']=='ON']
df = df[(df['Latitude']>43.64) & (df['Latitude']<43.68)]
df = df[(df['Longitude']<-79.36) & (df['Longitude']>-79.4)]
len(df)

71

In [25]:
sbux_ca = [{'type':'Point', 'coordinates':[long, lat]} for long in df['Longitude'] for lat in df['Latitude']]
len(sbux_ca)

5041

In [26]:
# Access main DataBase
client = MongoClient()
db = client.get_database("companies")

In [27]:
startups = db.top3
list(startups.find({},{'coord':1,'_id':0}))

[{'coord': {'type': 'Point', 'coordinates': [-73.567044, 45.498588]}},
 {'coord': {'type': 'Point', 'coordinates': [-80.546381, 43.474704]}},
 {'coord': {'type': 'Point', 'coordinates': [-123.103986, 49.283376]}},
 {'coord': {'type': 'Point', 'coordinates': [-79.549577, 43.76969]}},
 {'coord': {'type': 'Point', 'coordinates': [-123.138565, 49.263588]}},
 {'coord': {'type': 'Point', 'coordinates': [-79.5983626, 43.6587599]}},
 {'coord': {'type': 'Point', 'coordinates': [-79.338246, 43.7192415]}},
 {'coord': {'type': 'Point', 'coordinates': [-75.066111, 3.046111]}},
 {'coord': {'type': 'Point', 'coordinates': [-79.3953743, 43.6474554]}},
 {'coord': {'type': 'Point', 'coordinates': [-79.348037, 43.799511]}},
 {'coord': {'type': 'Point', 'coordinates': [-79.396805, 43.6501051]}},
 {'coord': {'type': 'Point', 'coordinates': [-73.566634, 45.506643]}},
 {'coord': {'type': 'Point', 'coordinates': [-106.346771, 56.130366]}},
 {'coord': {'type': 'Point', 'coordinates': [-73.572133, 45.513684]}},

In [28]:
for c in sbux_ca:
    query = {
                "coord":{
                    "$near":{
                        "$geometry":c,
                        "$maxDistance":5000
                    }
                }
            }
    res = startups.find(query)
    exec_selection = list(res)


In [29]:
exec_selection

[{'_id': ObjectId('5fbb5e6dd6feb44d924c0e32'),
  'name': 'BubbleShare',
  'permalink': 'bubbleshare',
  'crunchbase_url': 'http://www.crunchbase.com/company/bubbleshare',
  'homepage_url': 'http://www.bubbleshare.com',
  'blog_url': 'http://bubbleshare.blogspot.com/',
  'blog_feed_url': 'http://bubbleshare.blogspot.com/feeds/posts/default?alt=rss',
  'twitter_username': None,
  'category_code': 'mobile',
  'number_of_employees': None,
  'founded_year': 1991,
  'founded_month': 6,
  'founded_day': 28,
  'deadpooled_year': 2009,
  'deadpooled_month': 8,
  'deadpooled_day': None,
  'deadpooled_url': 'http://techcrunch.com/2009/08/13/long-journey-for-bubbleshare-ends-in-deadpool/',
  'tag_list': 'photos, photo-sharing',
  'alias_list': None,
  'email_address': '',
  'phone_number': '628568494016',
  'description': None,
  'created_at': 'Fri Apr 18 14:27:36 UTC 2008',
  'updated_at': 'Sat Jan 12 04:20:00 UTC 2013',
  'overview': '<p><a href="http://www.bubbleshare.com" title="BubbleShare" r

In [30]:
from folium import Map,CircleMarker

In [35]:
e_coord = [[ele ['coord']['coordinates'][1],ele ['coord']['coordinates'][0]] for ele in exec_selection]

In [38]:
m = Map(location=[43.6017, -79.68],zoom_start=15)


sites = [CircleMarker(location=ele, radius = 10) for ele in e_coord]
[ele.add_to(m) for ele in sites]

In [40]:
m